In [1]:

import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

import processing_io as pio
from torch_geometric.transforms import LineGraph

from torch_geometric.data import Data, Batch
import shapely.wkt as wkt
from tqdm import tqdm
import fiona
import os

import alphashape
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
from shapely.geometry import Point
import random

highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}
result_df_name = 'sim_output_1pm_capacity_reduction_10k_PRELIMINARY'
# result_df_name = 'sim_output_1pm_capacity_reduction_10k'

result_path = '../../../../data/datasets_simulation_outputs/' + result_df_name + '.pt'
string_is_for_1pm = "pop_1pm"

base_dir_sample_sim_input = '../../../../data/' + string_is_for_1pm + '_simulations/' + string_is_for_1pm + '_policies_combinations_with_normal_dist/'
subdirs_pattern = os.path.join(base_dir_sample_sim_input, 'output_networks_*')
subdirs = list(set(glob.glob(subdirs_pattern)))
subdirs.sort()

paris_inside_bvd_peripherique = "../../../../data/paris_inside_bvd_per/referentiel-comptages-edit.shp"
gdf_paris_inside_bvd_per = gpd.read_file(paris_inside_bvd_peripherique)
boundary_df = alphashape.alphashape(gdf_paris_inside_bvd_per, 435).exterior[0]
linear_ring_polygon = Polygon(boundary_df)

gdf_basecase_output_links = gpd.read_file('results/' + string_is_for_1pm + '_basecase_average_output_links.geojson')
gdf_basecase_average_mode_stats = pd.read_csv('results/' + string_is_for_1pm + '_basecase_average_mode_stats.csv', delimiter=';')
districts = gpd.read_file("../../../../data/visualisation/districts_paris.geojson")

## Abstract

This is further than process_output_of_simulations_with_all_output_links_and_eqasim_info.ipynb, as it also includes more input information.

Note that there is more than one strategy to deal with the fact that there are more than one district per link. We implement the strategy of stacking the information of all districts together. 
An alternative strategy would be to use the mean of the information of the districts.

Process the districts manually, so that each link belongs to at most 3 districts.

THIS IS OLD, MORE CURRENT ARE process_output_of_simulations_linegraph_has_nodes_for_districts_and_socio_infos.ipynb and process_output_of_simulations_linegraph_has_nodes_for_districts_and_socio_infos.ipynb.
DELETE THIS FILE ONCE THE NEW ONES ARE WORKING PERFECTLY.

## Process results

Process the outputs of the simulations for further usage by GNN.

In [19]:
def compute_close_homes(links_gdf_input:pd.DataFrame, information_gdf_input:pd.DataFrame, utm_crs:str, distance:int=50):
    links_gdf = links_gdf_input.copy()
    information_gdf = information_gdf_input.copy()
    close_places = []
    links_gdf_utm = links_gdf.to_crs(utm_crs)
    information_gdf_utm = information_gdf.to_crs(utm_crs)
    for i, row in tqdm(enumerate(links_gdf_utm.iterrows()), desc="Processing rows", unit="row"):
        buffer_utm = row[1].geometry.buffer(distance=distance)
        buffer = gpd.GeoSeries([buffer_utm], crs=utm_crs).to_crs(links_gdf_utm.crs)[0]
        matched_information = information_gdf_utm[information_gdf_utm.geometry.within(buffer)]
        socioprofessional_classes = matched_information['socioprofessional_class'].tolist()
        close_places.append((len(socioprofessional_classes), socioprofessional_classes))
    return close_places

def process_close_count_to_tensor(close_count_list: list):
    socio_professional_classes = [item[1] for item in close_count_list]
    unique_classes = set([2, 3, 4, 5, 6, 7, 8])
    class_to_index = {cls: idx for idx, cls in enumerate(unique_classes)}

    tensor_shape = (len(close_count_list), len(unique_classes))
    close_homes_tensor = torch.zeros(tensor_shape)

    for i, classes in enumerate(socio_professional_classes):
        for cls in classes:
            if cls in class_to_index:  # Ensure the class is in the predefined set
                close_homes_tensor[i, class_to_index[cls]] += 1
    
    close_homes_tensor_sparse = close_homes_tensor.to_sparse()
    return close_homes_tensor_sparse

# Read all network data into a dictionary of GeoDataFrames
def compute_result_dic_output_links():
    result_dic = {}
    base_network_no_policies = gdf_basecase_output_links
    result_dic["base_network_no_policies"] = base_network_no_policies
    for subdir in tqdm(subdirs, desc="Processing subdirs", unit="subdir"):
        # print(f'Accessing folder: {subdir}')
        # print(len(os.listdir(subdir)))
        networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
        for network in networks:
            file_path = os.path.join(subdir, network)
            policy_key = pio.create_policy_key_1pm(network)
            gdf_output_links = pio.read_output_links(file_path)
            if (gdf_output_links is not None):
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_output_links, column_to_extend='highway', new_column_name='highway')
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_output_links, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
                result_dic[policy_key] = gdf_extended
        break
    return result_dic

def calculate_averaged_results(trips_df):
    """Calculate average travel time and routed distance grouped by mode."""
    return trips_df.groupby('mode').agg(
        total_travel_time=('travel_time', 'mean'),
        total_routed_distance=('routed_distance', 'mean')
    ).reset_index()

def compute_result_dic_mode_stats(calculate_averaged_results):
    result_dic_mode_stats = {}
    result_dic_mode_stats["base_network_no_policies"] = gdf_basecase_average_mode_stats
    for subdir in tqdm(subdirs, desc="Processing subdirs", unit="subdir"):
        networks = [network for network in os.listdir(subdir) if not network.endswith(".DS_Store")]
        for network in networks:
            file_path = os.path.join(subdir, network)
            policy_key = pio.create_policy_key_1pm(network)
            df_mode_stats = pd.read_csv(file_path + '/eqasim_trips.csv', delimiter=';')
            averaged_results = calculate_averaged_results(df_mode_stats)
            if (averaged_results is not None):
                result_dic_mode_stats[policy_key] = averaged_results
        break
    return result_dic_mode_stats

def encode_modes(gdf):
    """Encode the 'modes' attribute based on specific strings."""
    modes_conditions = {
        'car': gdf['modes'].str.contains('car', case=False, na=False).astype(int),
        'bus': gdf['modes'].str.contains('bus', case=False, na=False).astype(int),
        'pt': gdf['modes'].str.contains('pt', case=False, na=False).astype(int),
        'train': gdf['modes'].str.contains('train', case=False, na=False).astype(int),
        'rail': gdf['modes'].str.contains('rail', case=False, na=False).astype(int),
        'subway': gdf['modes'].str.contains('subway', case=False, na=False).astype(int)
    }
    modes_encoded = pd.DataFrame(modes_conditions)
    return torch.tensor(modes_encoded.values, dtype=torch.float)

def get_dfs(base_dir:str):
    files = os.listdir(base_dir)
    for file in files:
        file_path = os.path.join(base_dir, file)
        base_name, ext = os.path.splitext(file)
        if base_name.startswith("idf_1pm_"):
            base_name = base_name.replace("idf_1pm_", "")
        var_name = base_name  # Start with the cleaned base name
    
        if file.endswith('.csv'):
            try:
                var_name = f"{var_name}_df"  
                globals()[var_name] = pd.read_csv(file_path, sep=";")
                print(f"Loaded CSV file: {file} into variable: {var_name}")
            except Exception as e:
                print(f"Error loading CSV file {file}: {e}")
            
        elif file.endswith('.gpkg'):
            try:
                var_name = f"{var_name}_gdf"  
                layers = fiona.listlayers(file_path)
                geodataframes = {layer: gpd.read_file(file_path, layer=layer, geometry = 'geometry', crs="EPSG:2154") for layer in layers}
                for layer, gdf in geodataframes.items():
                # print(f"Layer: {layer}")
                    gdf = gdf.to_crs(epsg=4326)
                    globals()[var_name] = gdf
                    print(f"Loaded GPKG file: {file} into variable: {var_name}")
            except Exception as e:
                print(f"Error loading CSV file {file}: {e}")
    homes_gdf = globals()["homes_gdf"]
    households_df = globals()["households_df"]
    persons_df = globals()["persons_df"]
    activities_gdf = globals()["activities_gdf"]
    trips_df = globals()["trips_gdf"]
    return homes_gdf, households_df, persons_df, activities_gdf, trips_df

def extract_start_end_points(geometry):
    if len(geometry.coords) != 2:
        raise ValueError("Linestring does not have exactly 2 elements.")
    return geometry.coords[0], geometry.coords[-1]

def get_close_trips_tensor(links_gdf_input, trips_gdf_input, utm_crs, distance):
    close_trips_count = compute_close_homes(links_gdf_input = links_gdf_input, information_gdf_input = trips_gdf_input, utm_crs = utm_crs, distance=distance)
    close_trips_count_tensor = process_close_count_to_tensor(close_trips_count)
    return close_trips_count, close_trips_count_tensor

def get_start_and_end_gdf(trips_with_socio, crs):
    trips_start = trips_with_socio.copy()
    trips_end = trips_with_socio.copy()

    trips_start_gdf = gpd.GeoDataFrame(
    trips_start, 
    geometry=gpd.points_from_xy(
        trips_start['start_point'].apply(lambda p: p[0]), 
        trips_start['start_point'].apply(lambda p: p[1])
    ), 
    crs=crs
)

    trips_end_gdf = gpd.GeoDataFrame(
    trips_end, 
    geometry=gpd.points_from_xy(
        trips_end['end_point'].apply(lambda p: p[0]), 
        trips_end['end_point'].apply(lambda p: p[1])
    ), 
    crs=crs
)
    return trips_start_gdf,trips_end_gdf

def process_centroid(geom_list):
    if not geom_list:  # Empty list
        return [np.nan, np.nan, np.nan]
    elif len(geom_list) == 1:
        return [geom_list[0], np.nan, np.nan]
    elif len(geom_list) == 2:
        return [geom_list[0], geom_list[1], np.nan]
    else:
        return [geom_list[0], geom_list[1], geom_list[2]]
    
def extract_point_coordinates(geom_list):
    coordinates = []
    for geom in geom_list:
        if isinstance(geom, Point):
            coordinates.append((geom.x, geom.y))
        else:
            coordinates.append((np.nan, np.nan))
    return coordinates

def process_value_list(perimeter_list):
    if not perimeter_list:  # Empty list
        return [np.nan, np.nan, np.nan]
    elif len(perimeter_list) == 1:
        return [perimeter_list[0], np.nan, np.nan]
    elif len(perimeter_list) == 2:
        return [perimeter_list[0], perimeter_list[1], np.nan]
    else:
        return [perimeter_list[0], perimeter_list[1], perimeter_list[2]]
    
def compute_district_2_information_counts(district_information_counts, column_to_filter_for):
    district_group_2_information_counts = {}
    for district, group in district_information_counts:        
        # ignore groups with more than one district here. 
        if len(district) == 1:
            total_counts = 0
            total_distributions = []
            counts = group[column_to_filter_for].values            
            for c in counts:
                total_counts += c[0]
                if c[1] is not None and len(c[1]) > 0:
                    total_distributions.extend(c[1])
            distribution_counts = [total_distributions.count(i) for i in range(2, 9)]   
            district_group_2_information_counts[district] = distribution_counts
    return district_group_2_information_counts, distribution_counts

def compute_district_2_information_tensor(district_2_information_counts, distribution_counts, gdf_input):
    district_home_counts_tensor = torch.zeros((len(gdf_input), 3, len(distribution_counts)), dtype=torch.float)
    nan_tensor = torch.full((len(distribution_counts),), float('nan'))

    for idx, row in gdf_input.iterrows():
        district_combination = row['district']
        district_combination_tuple = tuple(district_combination)
        if len(district_combination_tuple) == 0:
            district_home_counts_tensor[idx] = torch.stack([nan_tensor, nan_tensor, nan_tensor])
        elif len(district_combination_tuple) == 1:
            district_home_counts_tensor[idx] = torch.stack([torch.tensor(district_2_information_counts[district_combination_tuple]), nan_tensor, nan_tensor])
        elif len(district_combination_tuple) == 2:
            a, b = district_combination_tuple
            district_home_counts_tensor[idx] = torch.stack([torch.tensor(district_2_information_counts[(a,)]), torch.tensor(district_2_information_counts[(b,)]), nan_tensor])
        elif len(district_combination_tuple) == 3:
            a, b, c = district_combination_tuple
            district_home_counts_tensor[idx] = torch.stack([torch.tensor(district_2_information_counts[(a,)]), torch.tensor(district_2_information_counts[(b,)]), torch.tensor(district_2_information_counts[(c,)])])
        else:
            print("NOT OK!")
            print(district_combination_tuple)
    return district_home_counts_tensor

def preprocess_links(links_gdf):
    for index, row in links_gdf.iterrows():
        if len(row['district']) >= 4:
            row['district'].pop(random.randint(0, len(row['district']) - 1))
    return links_gdf

In [3]:
result_dic_output_links = compute_result_dic_output_links()
result_dic_mode_stats = compute_result_dic_mode_stats(calculate_averaged_results)
base_gdf = result_dic_output_links["base_network_no_policies"]
links_gdf_base = gpd.GeoDataFrame(base_gdf, geometry='geometry')
links_gdf_base.crs = "EPSG:2154"  # Assuming the original CRS is EPSG:2154
links_gdf_base.to_crs("EPSG:4326", inplace=True)
districts['district_centroid'] = districts['geometry'].centroid
links_gdf_with_districts = gpd.sjoin(links_gdf_base, districts, how='left', op='intersects')

# Group by edge and aggregate the district names
links_gdf_with_districts = links_gdf_with_districts.groupby('link').agg({
    'from_node': 'first',
    'to_node': 'first',
    'length': 'first',
    'freespeed': 'first',
    'capacity': 'first',
    'lanes': 'first',
    'modes': 'first',
    'vol_car': 'first',
    'highway': 'first',
    'geometry': 'first',
    'c_ar': lambda x: list(x.dropna()),
    'district_centroid': lambda x: list(x.dropna()),
    'perimetre': lambda x: list(x.dropna()),
    'surface': lambda x: list(x.dropna()),
}).reset_index()
gdf_now = gpd.GeoDataFrame(links_gdf_with_districts, geometry='geometry', crs=links_gdf_base.crs)
gdf_now = gdf_now.rename(columns={'c_ar': 'district', 'perimetre': 'district_perimeter', 'surface': 'district_surface'})
links_gdf_final = gdf_now.copy()
links_gdf_final = preprocess_links(links_gdf_final)

Processing subdirs:   0%|          | 0/92 [00:01<?, ?subdir/s]
/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_23290/1320727273.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  districts['district_centroid'] = districts['geometry'].centroid
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/Users/elenanatterer/anaconda3/envs/Paris_Analysis/lib/python3.10/site-packages/pandas/core/reshape/merge.py:1203: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():


In [5]:
max_district_length = max(links_gdf_final['district'].apply(len))
print(f"The maximum length of the list in 'district' in links_gdf_final is: {max_district_length}")

The maximum length of the list in 'district' in links_gdf_final is: 3


In [6]:
base_dir_sample_sim_input = '../../../../data/pop_1pm_simulations/idf_1pm/' 
homes_gdf, households_df, persons_df, activities_gdf, trips_df = get_dfs(base_dir=base_dir_sample_sim_input)

Loaded CSV file: idf_1pm_persons.csv into variable: persons_df
Loaded GPKG file: idf_1pm_commutes.gpkg into variable: commutes_gdf
Loaded CSV file: idf_1pm_households.csv into variable: households_df
Loaded CSV file: idf_1pm_trips.csv into variable: trips_df
Loaded CSV file: idf_1pm_activities.csv into variable: activities_df
Loaded CSV file: idf_1pm_vehicle_types.csv into variable: vehicle_types_df
Loaded GPKG file: idf_1pm_trips.gpkg into variable: trips_gdf
Loaded GPKG file: idf_1pm_activities.gpkg into variable: activities_gdf
Loaded CSV file: idf_1pm_vehicles.csv into variable: vehicles_df
Loaded GPKG file: idf_1pm_homes.gpkg into variable: homes_gdf


In [7]:
population_df = pd.read_csv("intermediate_results/population.csv")

sorted_population_df = population_df.sort_values(by="id")
sorted_persons_df = persons_df.sort_values(by="person_id")
merged_df = pd.merge(sorted_persons_df, sorted_population_df, left_on="person_id", right_on="id")
removed_some_columns = merged_df.copy()
removed_some_columns = removed_some_columns.drop(columns=['employed_y', 'hasPtSubscription', 'householdId', 'sex_y', 'htsPersonId', 'censusPersonId', 'hasLicense', 'id', 'age_y'])
updated_persons = removed_some_columns.copy()
persons_with_geospatial_information = homes_gdf.merge(updated_persons, on='household_id', how='right')

if not isinstance(persons_with_geospatial_information, gpd.GeoDataFrame):
    persons_with_geospatial_information = gpd.GeoDataFrame(persons_with_geospatial_information, geometry=gpd.points_from_xy(persons_with_geospatial_information.longitude, persons_with_geospatial_information.latitude), crs= links_gdf_final.crs)

utm_crs = 'EPSG:32631'  # UTM zone 31N

In [8]:
# DEAL WITH TRIPS

trips_with_socio = trips_df.merge(persons_with_geospatial_information[['person_id', 'socioprofessional_class']], on='person_id', how='left')
trips_with_socio['start_point'] = trips_with_socio['geometry'].apply(lambda geom: extract_start_end_points(geom)[0])
trips_with_socio['end_point'] = trips_with_socio['geometry'].apply(lambda geom: extract_start_end_points(geom)[1])

trips_start_gdf, trips_end_gdf = get_start_and_end_gdf(trips_with_socio=trips_with_socio, crs=links_gdf_final.crs)

# Create tensors for each combination of "preceding_purpose" and "following_purpose"
unique_purposes = trips_with_socio['preceding_purpose'].unique()
close_start_trips_tensor_dict = {}
close_start_trips_dict = {}
close_end_trips_tensor_dict = {}
close_end_trips_dict = {}

for preceding_purpose in tqdm(unique_purposes, desc="Processing preceding purposes", unit="purpose"):
    for following_purpose in tqdm(unique_purposes, desc="Processing following purposes", unit="purpose"):
        if preceding_purpose != following_purpose:
            filtered_trips = trips_with_socio[(trips_with_socio['preceding_purpose'] == preceding_purpose) & (trips_with_socio['following_purpose'] == following_purpose)]
            if not filtered_trips.empty:
                filtered_trips_start_gdf, filtered_trips_end_gdf = get_start_and_end_gdf(trips_with_socio=filtered_trips, crs=links_gdf_final.crs)
        
                tensor_key = f"{preceding_purpose}_{following_purpose}"
                string_trips_start = "trips_start_" + tensor_key
                string_trips_end = "trips_end_" + tensor_key
                
                close_start_trips, close_start_trips_tensor = get_close_trips_tensor(links_gdf_input=links_gdf_final, trips_gdf_input=filtered_trips_start_gdf, utm_crs=utm_crs, distance=50)        
                close_start_trips_tensor_dict[tensor_key] = close_start_trips_tensor
                close_start_trips_dict[tensor_key] = close_start_trips
                links_gdf_final[string_trips_start ] = close_start_trips
                
                close_end_trips, close_end_trips_tensor = get_close_trips_tensor(links_gdf_input=links_gdf_final, trips_gdf_input=filtered_trips_end_gdf, utm_crs=utm_crs, distance=50)
                close_end_trips_tensor_dict[tensor_key] = close_end_trips_tensor
                close_end_trips_dict[tensor_key] = close_end_trips
                links_gdf_final[string_trips_end] = close_end_trips

for key, tensor in close_start_trips_tensor_dict.items():
    print(f"Size of tensor for {key}: {tensor.size()}")

Processing preceding purposes:   0%|          | 0/6 [00:00<?, ?purpose/s]


































































































































































































































































































































































































































































































































































































































Processing rows: 31216row [01:05, 479.84row/s]















































































































































































































































































Size of tensor for home_leisure: torch.Size([31216, 7])
Size of tensor for home_work: torch.Size([31216, 7])
Size of tensor for home_shop: torch.Size([31216, 7])
Size of tensor for home_other: torch.Size([31216, 7])
Size of tensor for home_education: torch.Size([31216, 7])
Size of tensor for leisure_home: torch.Size([31216, 7])
Size of tensor for leisure_work: torch.Size([31216, 7])
Size of tensor for leisure_shop: torch.Size([31216, 7])
Size of tensor for leisure_other: torch.Size([31216, 7])
Size of tensor for leisure_education: torch.Size([31216, 7])
Size of tensor for work_home: torch.Size([31216, 7])
Size of tensor for work_leisure: torch.Size([31216, 7])
Size of tensor for work_shop: torch.Size([31216, 7])
Size of tensor for work_other: torch.Size([31216, 7])
Size of tensor for shop_home: torch.Size([31216, 7])
Size of tensor for shop_leisure: torch.Size([31216, 7])
Size of tensor for shop_work: torch.Size([31216, 7])
Size of tensor for shop_other: torch.Size([31216, 7])
Size of 

In [18]:
# for key, tensor in close_end_trips_tensor_dict.items():
#     print(f"Size of tensor for {key}: {tensor.size()}")
    
for key, tensor in close_end_trips_dict.items():
    print(f"Size of tensor for {key}: {len(tensor)}")

Size of tensor for home_leisure: 31216
Size of tensor for home_work: 31216
Size of tensor for home_shop: 31216
Size of tensor for home_other: 31216
Size of tensor for home_education: 31216
Size of tensor for leisure_home: 31216
Size of tensor for leisure_work: 31216
Size of tensor for leisure_shop: 31216
Size of tensor for leisure_other: 31216
Size of tensor for leisure_education: 31216
Size of tensor for work_home: 31216
Size of tensor for work_leisure: 31216
Size of tensor for work_shop: 31216
Size of tensor for work_other: 31216
Size of tensor for shop_home: 31216
Size of tensor for shop_leisure: 31216
Size of tensor for shop_work: 31216
Size of tensor for shop_other: 31216
Size of tensor for shop_education: 31216
Size of tensor for other_home: 31216
Size of tensor for other_leisure: 31216
Size of tensor for other_work: 31216
Size of tensor for other_shop: 31216
Size of tensor for other_education: 31216
Size of tensor for education_home: 31216
Size of tensor for education_leisure: 3

In [9]:
# DEAL WITH HOMES

close_homes_count_normal = compute_close_homes(links_gdf_input = links_gdf_final, information_gdf_input = persons_with_geospatial_information, utm_crs = utm_crs)
links_gdf_final['close_homes_count'] = close_homes_count_normal
close_homes_tensor = process_close_count_to_tensor(close_homes_count_normal)

Processing rows: 31216row [01:03, 493.14row/s]


In [10]:
# DEAL WITH ACTIVITIES

activities_with_socio = activities_gdf.merge(persons_with_geospatial_information[['household_id', 'socioprofessional_class']], on='household_id', how='left')
grouped_activities = activities_with_socio.groupby('purpose')
activities_by_purpose = {purpose: group.reset_index(drop=True) for purpose, group in grouped_activities}
activities_by_purpose_tensor = {}
for purpose, activities in activities_by_purpose.items():
    close_activities_count_purpose = f"close_activities_count_{purpose}"
    close_activity_count = compute_close_homes(links_gdf_input=links_gdf_final, information_gdf_input=activities, utm_crs=utm_crs)
    links_gdf_final[close_activities_count_purpose] = close_activity_count
    activities_by_purpose_tensor[purpose] = process_close_count_to_tensor(close_activity_count)    

Processing rows: 31216row [01:04, 481.08row/s]
Processing rows: 31216row [01:53, 274.91row/s]
Processing rows: 31216row [01:09, 447.99row/s]
Processing rows: 31216row [01:06, 470.98row/s]
Processing rows: 31216row [01:06, 468.97row/s]
Processing rows: 31216row [01:09, 446.71row/s]


# Deal with districts

In [11]:
# DISTRICTS GENERAL 

links_gdf_final['districts_tuple'] = links_gdf_final['district'].apply(lambda x: tuple(x))
district_tuples = links_gdf_final.groupby('districts_tuple')

In [20]:
# DISTRICT CENTROIDS

centroid_distances = np.array([
    process_centroid(geom_list)
    for geom_list in links_gdf_final['district_centroid']
])

# Process the centroids
centroid_distance_with_coordinates = np.array([
    extract_point_coordinates(geom_list)
    for geom_list in centroid_distances
])
district_centroids_tensor = torch.tensor(centroid_distance_with_coordinates, dtype=torch.float)

In [21]:
# DISTRICT PERIMETER AND SURFACE

district_perimeters = np.array([
    process_value_list(perimeter_list)
    for perimeter_list in links_gdf_final['district_perimeter']
])
district_surfaces = np.array([
    process_value_list(surface_list)
    for surface_list in links_gdf_final['district_surface']
])

district_perimeters_tensor = torch.tensor(district_perimeters, dtype=torch.float)
district_surfaces_tensor = torch.tensor(district_surfaces, dtype=torch.float)

In [22]:
# DISTRICT HOME COUNTS 

district_2_home_counts, distribution_counts = compute_district_2_information_counts(district_information_counts=district_tuples, column_to_filter_for = 'close_homes_count')
district_home_counts_tensor = compute_district_2_information_tensor(district_2_information_counts=district_2_home_counts, distribution_counts=distribution_counts, gdf_input=links_gdf_final)

In [23]:
# DISTRICT ACTIVITIES COUNTS

activity_2_district_tensor = {}
for purpose in activities_by_purpose_tensor.keys():
    district_2_activity_counts, distribution_counts_activity = compute_district_2_information_counts(district_information_counts=district_tuples, column_to_filter_for='close_activities_count_' + purpose)
    district_activitiy_counts_tensor = compute_district_2_information_tensor(district_2_information_counts=district_2_activity_counts, distribution_counts=distribution_counts_activity, gdf_input=links_gdf_final)
    activity_2_district_tensor[purpose] = district_activitiy_counts_tensor

In [24]:
indices = torch.nonzero(~torch.isnan(activity_2_district_tensor['education'][:, 1, 0])).squeeze()
indices

tensor([  109,   184,   185,  ..., 31182, 31186, 31208])

In [25]:
links_gdf_final.columns

Index(['link', 'from_node', 'to_node', 'length', 'freespeed', 'capacity',
       'lanes', 'modes', 'vol_car', 'highway', 'geometry', 'district',
       'district_centroid', 'district_perimeter', 'district_surface',
       'trips_start_home_leisure', 'trips_end_home_leisure',
       'trips_start_home_work', 'trips_end_home_work', 'trips_start_home_shop',
       'trips_end_home_shop', 'trips_start_home_other', 'trips_end_home_other',
       'trips_start_home_education', 'trips_end_home_education',
       'trips_start_leisure_home', 'trips_end_leisure_home',
       'trips_start_leisure_work', 'trips_end_leisure_work',
       'trips_start_leisure_shop', 'trips_end_leisure_shop',
       'trips_start_leisure_other', 'trips_end_leisure_other',
       'trips_start_leisure_education', 'trips_end_leisure_education',
       'trips_start_work_home', 'trips_end_work_home',
       'trips_start_work_leisure', 'trips_end_work_leisure',
       'trips_start_work_shop', 'trips_end_work_shop',
       'tri

In [26]:
# DISTRICT TRIPS START AND END

unique_purposes = trips_with_socio['preceding_purpose'].unique()
district_close_start_trips_tensor_dict = {}
district_close_end_trips_tensor_dict = {}

for preceding_purpose in tqdm(unique_purposes, desc="Processing preceding purposes", unit="purpose"):
    for following_purpose in tqdm(unique_purposes, desc="Processing following purposes", unit="purpose"):
        if preceding_purpose != following_purpose:
            filtered_trips = trips_with_socio[(trips_with_socio['preceding_purpose'] == preceding_purpose) & (trips_with_socio['following_purpose'] == following_purpose)]
            if not filtered_trips.empty:
                tensor_key = f"{preceding_purpose}_{following_purpose}"
                print(tensor_key)

                string_trips_start_purpose = "trips_start_" + tensor_key
                string_trips_end_purpose = "trips_start_" + tensor_key
                
                district_2_start_trips, start_trip_distributions = compute_district_2_information_counts(district_information_counts=district_tuples, column_to_filter_for = string_trips_start_purpose)
                district_trip_starts_tensor = compute_district_2_information_tensor(district_2_information_counts=district_2_start_trips, distribution_counts=start_trip_distributions, gdf_input=links_gdf_final)
                district_close_start_trips_tensor_dict[string_trips_start_purpose] = district_trip_starts_tensor
                
                district_2_end_trips, end_trip_distributions = compute_district_2_information_counts(district_information_counts=district_tuples, column_to_filter_for = string_trips_end_purpose)
                district_trip_end_tensor = compute_district_2_information_tensor(district_2_information_counts=district_2_end_trips, distribution_counts=end_trip_distributions, gdf_input=links_gdf_final)
                district_close_end_trips_tensor_dict[string_trips_end_purpose] = district_trip_end_tensor

Processing preceding purposes:   0%|          | 0/6 [00:00<?, ?purpose/s]

home


leisure
home_leisure


work
home_work


shop
home_shop


...

leisure
education_leisure


work
education_work


shop
education_shop


other
education_other


Processing preceding purposes: 100%|██████████| 6/6 [01:00<00:00, 10.08s/purpose]


In [27]:
indices = torch.nonzero(~torch.isnan(district_close_start_trips_tensor_dict['trips_start_home_leisure'][:, 1, 0])).squeeze()
indices

tensor([  109,   184,   185,  ..., 31182, 31186, 31208])

In [28]:
# PROCESS LINK GEOMETRIES

edge_midpoints = np.array([((geom.coords[0][0] + geom.coords[-1][0]) / 2, 
                                    (geom.coords[0][1] + geom.coords[-1][1]) / 2) 
                                for geom in links_gdf_final.geometry])

nodes = pd.concat([links_gdf_final['from_node'], links_gdf_final['to_node']]).unique()
node_to_idx = {node: idx for idx, node in enumerate(nodes)}
links_gdf_final['from_idx'] = links_gdf_final['from_node'].map(node_to_idx)
links_gdf_final['to_idx'] = links_gdf_final['to_node'].map(node_to_idx)
edges_base = links_gdf_final[['from_idx', 'to_idx']].values
edge_midpoint_tensor = torch.tensor(edge_midpoints, dtype=torch.float)

# Initialize start and end points
start_points = np.array([geom.coords[0] for geom in links_gdf_final.geometry])
end_points = np.array([geom.coords[-1] for geom in links_gdf_final.geometry])

# Convert to tensors
edge_start_point_tensor = torch.tensor(start_points, dtype=torch.float)
edge_end_point_tensor = torch.tensor(end_points, dtype=torch.float)

edge_start_end_tensor = torch.stack((edge_start_point_tensor, edge_end_point_tensor), dim=1)

## Analyze results and plot

In [29]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [30]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [31]:
# Stack all entries of trips in one tensor
close_start_trips_tensors = [tensor.to_dense() for tensor in close_start_trips_tensor_dict.values()]
stacked_close_start_trips_tensor = torch.cat(close_start_trips_tensors, dim=1)

close_end_trips_tensors = [tensor.to_dense() for tensor in close_end_trips_tensor_dict.values()]
stacked_close_end_trips_tensor = torch.cat(close_end_trips_tensors, dim=1)

district_start_trips_tensors = [tensor.to_dense() for tensor in district_close_start_trips_tensor_dict.values()]
stacked_district_start_trips_tensor = torch.cat(district_start_trips_tensors, dim=1)

district_end_trips_tensors = [tensor.to_dense() for tensor in district_close_end_trips_tensor_dict.values()]
stacked_district_end_trips_tensor = torch.cat(district_end_trips_tensors, dim=1)

In [2]:
def process_result_dic(result_dic, result_dic_mode_stats):
    datalist = []
    linegraph_transformation = LineGraph()
    
    edge_start_point_t = edge_start_point_tensor.to_dense()
    edge_end_point_t = edge_end_point_tensor.to_dense()
    edge_start_end_point_t = edge_start_end_tensor.to_dense()
    edge_midpoint_t = edge_midpoint_tensor.to_dense()
    
    base_network_no_policies = result_dic.get("base_network_no_policies")
    vol_base_case = base_network_no_policies['vol_car'].values
    capacity_base_case = base_network_no_policies['capacity'].values
    length_base_case = base_network_no_policies['length'].values
    freespeed_base_case = base_network_no_policies['freespeed'].values
    modes_base_case = encode_modes(base_network_no_policies)
    close_homes = close_homes_tensor.to_dense()
    
    # stack in one tensor
    activities_home = activities_by_purpose_tensor['home'].to_dense()
    activities_work = activities_by_purpose_tensor['work'].to_dense()
    activities_education = activities_by_purpose_tensor['education'].to_dense()
    activities_shop = activities_by_purpose_tensor['shop'].to_dense()
    activities_leisure = activities_by_purpose_tensor['leisure'].to_dense()
    activities_other = activities_by_purpose_tensor['other'].to_dense()
    
    district_centroids = district_centroids_tensor.to_dense()
    district_perimeters = district_perimeters_tensor.to_dense()
    district_surfaces = district_surfaces_tensor.to_dense()
    district_home_counts = district_home_counts_tensor.to_dense()
    
    # stack in one tensor
    district_activities_home = activity_2_district_tensor['home'].to_dense()
    district_activities_work = activity_2_district_tensor['work'].to_dense()
    district_activities_education = activity_2_district_tensor['education'].to_dense()
    district_activities_shop = activity_2_district_tensor['shop'].to_dense()
    district_activities_leisure = activity_2_district_tensor['leisure'].to_dense()
    district_activities_other = activity_2_district_tensor['other'].to_dense()
    
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x)
    # data = Data(edge_index=edge_index, x=x, pos=edge_positions_tensor)
    linegraph_data = linegraph_transformation(data)

    for key, df in tqdm(result_dic.items(), desc="Processing result_dic", unit="dataframe"):        
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = gpd.GeoDataFrame(df, geometry='geometry')
            gdf.crs = "EPSG:2154"  
            gdf.to_crs("EPSG:4326", inplace=True)
            capacities_new = gdf['capacity'].values
            capacity_reduction = capacities_new - capacity_base_case
            highway = gdf['highway'].apply(lambda x: highway_mapping.get(x, -1)).values
            length = gdf['length'].values
            freespeed= gdf['freespeed'].values
            modes = encode_modes(gdf)
            edge_car_volume_difference = gdf['vol_car'].values - vol_base_case
            target_values = torch.tensor(edge_car_volume_difference, dtype=torch.float).unsqueeze(1)

            tensors_edge_information = [
                vol_base_case, capacity_base_case, capacities_new, capacity_reduction, 
                highway, length, freespeed, length_base_case, freespeed_base_case, 
                modes, modes_base_case, 
            ]
            
            tensors_districts_information = []
            
            tensors_with_sociographic_information = [close_homes, 
                activities_home, activities_work, activities_education, activities_shop, activities_leisure, activities_other,
                stacked_close_start_trips_tensor, stacked_close_end_trips_tensor]
                
            tensors_districts_with_sociographic_information = [
                district_home_counts,
                district_activities_home, district_activities_work, district_activities_education, district_activities_shop, district_activities_leisure, district_activities_other,
                stacked_district_start_trips_tensor, stacked_district_end_trips_tensor]
            
            linegraph_x = torch.tensor(np.column_stack(tensors_edge_information, tensors_with_sociographic_information), dtype=torch.float)
            linegraph_x_districts = torch.tensor(np.column_stack(tensors_districts_information, tensors_districts_with_sociographic_information), dtype=torch.float)
            # stack linegraph_x and linegraph_x_districts
            
            
            linegraph_pos = torch.tensor(np.column_stack(edge_start_end_point_t, edge_start_point_t, edge_end_point_t, edge_midpoint_t, district_centroids), dtype=torch.float)

            # Print shapes for debugging
            for i, t in enumerate(tensors_edge_information):
                print(f"Shape of tensor {i}: {t.shape}")

            linegraph_data.x = linegraph_x
            linegraph_data.pos = linegraph_pos
            linegraph_data.y = target_values
            
            df_mode_stats = result_dic_mode_stats.get(key)
            if df_mode_stats is not None:
                numeric_cols = df_mode_stats.select_dtypes(include=[np.number]).columns
                mode_stats_numeric = df_mode_stats[numeric_cols].astype(float)
                mode_stats_tensor = torch.tensor(mode_stats_numeric.values, dtype=torch.float)
                linegraph_data.mode_stats = mode_stats_tensor
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
            else:
                print("Invalid line graph data")
    data_dict_list = [{'x': lg_data.x, 'edge_index': lg_data.edge_index, 'pos': lg_data.pos, 'y': lg_data.y, 'graph_attr': lg_data.mode_stats} for lg_data in datalist]
    return data_dict_list

data_processed = process_result_dic(result_dic=result_dic_output_links, result_dic_mode_stats=result_dic_mode_stats)
torch.save(data_processed, result_path)

NameError: name 'result_dic_output_links' is not defined

In [37]:
def process_result_dic(result_dic, result_dic_mode_stats):
    datalist = []
    linegraph_transformation = LineGraph()
    
    edge_start_point_t = edge_start_point_tensor.to_dense()
    edge_end_point_t = edge_end_point_tensor.to_dense()
    edge_start_end_point_t = edge_start_end_tensor.to_dense()
    edge_midpoint_t = edge_midpoint_tensor.to_dense()
    
    base_network_no_policies = result_dic.get("base_network_no_policies")
    vol_base_case = base_network_no_policies['vol_car'].values
    capacity_base_case = base_network_no_policies['capacity'].values
    length_base_case = base_network_no_policies['length'].values
    freespeed_base_case = base_network_no_policies['freespeed'].values
    modes_base_case = encode_modes(base_network_no_policies)
    close_homes = close_homes_tensor.to_dense()
    
    # stack in one tensor
    activities_home = activities_by_purpose_tensor['home'].to_dense()
    activities_work = activities_by_purpose_tensor['work'].to_dense()
    activities_education = activities_by_purpose_tensor['education'].to_dense()
    activities_shop = activities_by_purpose_tensor['shop'].to_dense()
    activities_leisure = activities_by_purpose_tensor['leisure'].to_dense()
    activities_other = activities_by_purpose_tensor['other'].to_dense()
    
    district_centroids = district_centroids_tensor.to_dense()
    district_perimeters = district_perimeters_tensor.to_dense()
    district_surfaces = district_surfaces_tensor.to_dense()
    district_home_counts = district_home_counts_tensor.to_dense()
    
    # stack in one tensor
    district_activities_home = activity_2_district_tensor['home'].to_dense()
    district_activities_work = activity_2_district_tensor['work'].to_dense()
    district_activities_education = activity_2_district_tensor['education'].to_dense()
    district_activities_shop = activity_2_district_tensor['shop'].to_dense()
    district_activities_leisure = activity_2_district_tensor['leisure'].to_dense()
    district_activities_other = activity_2_district_tensor['other'].to_dense()
    
    edge_index = torch.tensor(edges_base, dtype=torch.long).t().contiguous()
    x = torch.zeros((len(nodes), 1), dtype=torch.float)
    data = Data(edge_index=edge_index, x=x)
    # data = Data(edge_index=edge_index, x=x, pos=edge_positions_tensor)
    linegraph_data = linegraph_transformation(data)

    for key, df in tqdm(result_dic.items(), desc="Processing result_dic", unit="dataframe"):        
        if isinstance(df, pd.DataFrame) and key != "base_network_no_policies":
            gdf = gpd.GeoDataFrame(df, geometry='geometry')
            gdf.crs = "EPSG:2154"  
            gdf.to_crs("EPSG:4326", inplace=True)
            capacities_new = gdf['capacity'].values
            capacity_reduction = capacities_new - capacity_base_case
            highway = gdf['highway'].apply(lambda x: highway_mapping.get(x, -1)).values
            length = gdf['length'].values
            freespeed= gdf['freespeed'].values
            modes = encode_modes(gdf)
            edge_car_volume_difference = gdf['vol_car'].values - vol_base_case
            target_values = torch.tensor(edge_car_volume_difference, dtype=torch.float).unsqueeze(1)

            tensors_edge_information = [
                vol_base_case, capacity_base_case, capacities_new, capacity_reduction, 
                highway, length, freespeed, length_base_case, freespeed_base_case, 
                modes, modes_base_case, 
            ]
            
            tensors_districts_information = []
            
            tensors_with_sociographic_information = [close_homes, 
                activities_home, activities_work, activities_education, activities_shop, activities_leisure, activities_other,
                stacked_close_start_trips_tensor, stacked_close_end_trips_tensor]
                
             tensors_districts_with_sociographic_information = [
                district_home_counts,
                district_activities_home, district_activities_work, district_activities_education, district_activities_shop, district_activities_leisure, district_activities_other,
                stacked_district_start_trips_tensor, stacked_district_end_trips_tensor]
            
            linegraph_x = torch.tensor(np.column_stack(tensors_edge_information, tensors_with_sociographic_information), dtype=torch.float)
            linegraph_x_districts = torch.tensor(np.column_stack(tensors_districts_information, tensors_districts_with_sociographic_information), dtype=torch.float)
            # stack linegraph_x and linegraph_x_districts
            
            
            linegraph_pos = torch.tensor(np.column_stack(edge_start_end_point_t, edge_start_point_t, edge_end_point_t, edge_midpoint_t, district_centroids), dtype=torch.float)

            # Print shapes for debugging
            for i, t in enumerate(tensors_edge_information):
                print(f"Shape of tensor {i}: {t.shape}")

            linegraph_data.x = linegraph_x
            linegraph_data.pos = linegraph_pos
            linegraph_data.y = target_values
            
            df_mode_stats = result_dic_mode_stats.get(key)
            if df_mode_stats is not None:
                numeric_cols = df_mode_stats.select_dtypes(include=[np.number]).columns
                mode_stats_numeric = df_mode_stats[numeric_cols].astype(float)
                mode_stats_tensor = torch.tensor(mode_stats_numeric.values, dtype=torch.float)
                linegraph_data.mode_stats = mode_stats_tensor
            if linegraph_data.validate(raise_on_error=True):
                datalist.append(linegraph_data)
            else:
                print("Invalid line graph data")
    data_dict_list = [{'x': lg_data.x, 'edge_index': lg_data.edge_index, 'pos': lg_data.pos, 'y': lg_data.y, 'graph_attr': lg_data.mode_stats} for lg_data in datalist]
    return data_dict_list

data_processed = process_result_dic(result_dic=result_dic_output_links, result_dic_mode_stats=result_dic_mode_stats)
torch.save(data_processed, result_path)

Processing result_dic:   3%|▎         | 2/79 [00:00<00:11,  6.99dataframe/s]

Shape of tensor 0: (31216, 2, 2)
Shape of tensor 1: (31216, 2)
Shape of tensor 2: (31216, 2)
Shape of tensor 3: (31216, 2)
Shape of tensor 4: (31216, 3, 2)
Shape of reshaped tensor 0: (31216, 4)
Shape of reshaped tensor 1: (31216, 2)
Shape of reshaped tensor 2: (31216, 2)
Shape of reshaped tensor 3: (31216, 2)
Shape of reshaped tensor 4: (31216, 6)


Processing result_dic:   4%|▍         | 3/79 [00:00<00:15,  4.89dataframe/s]

Shape of tensor 0: (31216, 2, 2)
Shape of tensor 1: (31216, 2)
Shape of tensor 2: (31216, 2)
Shape of tensor 3: (31216, 2)
Shape of tensor 4: (31216, 3, 2)
Shape of reshaped tensor 0: (31216, 4)
Shape of reshaped tensor 1: (31216, 2)
Shape of reshaped tensor 2: (31216, 2)
Shape of reshaped tensor 3: (31216, 2)
Shape of reshaped tensor 4: (31216, 6)


Processing result_dic:   5%|▌         | 4/79 [00:00<00:15,  4.84dataframe/s]

...

Shape of tensor 0: (31216, 2, 2)
Shape of tensor 1: (31216, 2)
Shape of tensor 2: (31216, 2)
Shape of tensor 3: (31216, 2)
Shape of tensor 4: (31216, 3, 2)
Shape of reshaped tensor 0: (31216, 4)
Shape of reshaped tensor 1: (31216, 2)
Shape of reshaped tensor 2: (31216, 2)
Shape of reshaped tensor 3: (31216, 2)
Shape of reshaped tensor 4: (31216, 6)


Processing result_dic:  99%|█████████▊| 78/79 [00:17<00:00,  4.82dataframe/s]

Shape of tensor 0: (31216, 2, 2)
Shape of tensor 1: (31216, 2)
Shape of tensor 2: (31216, 2)
Shape of tensor 3: (31216, 2)
Shape of tensor 4: (31216, 3, 2)
Shape of reshaped tensor 0: (31216, 4)
Shape of reshaped tensor 1: (31216, 2)
Shape of reshaped tensor 2: (31216, 2)
Shape of reshaped tensor 3: (31216, 2)
Shape of reshaped tensor 4: (31216, 6)


Processing result_dic: 100%|██████████| 79/79 [00:18<00:00,  4.37dataframe/s]

Shape of tensor 0: (31216, 2, 2)
Shape of tensor 1: (31216, 2)
Shape of tensor 2: (31216, 2)
Shape of tensor 3: (31216, 2)
Shape of tensor 4: (31216, 3, 2)
Shape of reshaped tensor 0: (31216, 4)
Shape of reshaped tensor 1: (31216, 2)
Shape of reshaped tensor 2: (31216, 2)
Shape of reshaped tensor 3: (31216, 2)
Shape of reshaped tensor 4: (31216, 6)


In [38]:
data_processed[0]['x']

{'x': tensor([[6.6275e+00, 4.8000e+02, 4.8000e+02,  ...,        nan,        nan,
                 nan],
         [9.6078e+00, 4.8000e+02, 4.8000e+02,  ...,        nan,        nan,
                 nan],
         [2.4902e+00, 9.6000e+02, 9.6000e+02,  ...,        nan,        nan,
                 nan],
         ...,
         [0.0000e+00, 7.9992e+03, 7.9992e+03,  ...,        nan,        nan,
                 nan],
         [0.0000e+00, 7.9992e+03, 7.9992e+03,  ...,        nan,        nan,
                 nan],
         [0.0000e+00, 7.9992e+03, 7.9992e+03,  ...,        nan,        nan,
                 nan]], dtype=torch.float64),
 'edge_index': tensor([[    0,     1,     1,  ..., 31138, 31139, 31139],
         [   19, 10935, 10936,  ..., 30278, 31138, 31139]]),
 'pos': tensor([[ 2.3387, 48.8518,  2.3385,  ...,     nan,     nan,     nan],
         [ 2.3387, 48.8524,  2.3387,  ...,     nan,     nan,     nan],
         [ 2.3387, 48.8523,  2.3387,  ...,     nan,     nan,     nan],
         .

## Save for further processing with GNN

In [ ]:
# data_processed_single_districts = pio.process_result_dic(result_dic_single_districts)
# torch.save(data_processed_single_districts, result_path + '_single_districts.pt')

In [ ]:
torch.save(data_processed, result_path)

In [ ]:
# plt.figure(figsize=(10, 6))
# plt.scatter(persons_with_homes.geometry.x, persons_with_homes.geometry.y, s=1, color='blue', alpha=0.5)
# plt.scatter(persons_with_home_within_linear_ring.geometry.x, persons_with_home_within_linear_ring.geometry.y, s=1, color='red', alpha=0.5)
# plt.title('Locations of Persons with Homes')
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.show()

# from shapely.geometry import LineString
# from shapely.geometry import MultiPolygon
# import matplotlib.pyplot as plt

# # Create a LineString
# line = LineString([(10, 10), (20, 10)])

# # Create a buffer around the line
# buffered_line = line.buffer(2, cap_style="round")

# # Plot the original line and the buffered area
# plt.figure(figsize=(8, 6))
# x, y = line.xy
# plt.plot(x, y, color='blue', label='Original Line')
# if isinstance(buffered_line, MultiPolygon):
#     for polygon in buffered_line:
#         x, y = polygon.exterior.xy
#         plt.fill(x, y, alpha=0.5, color='lightblue', label='Buffered Area')
# else:
#     x, y = buffered_line.exterior.xy
#     plt.fill(x, y, alpha=0.5, color='lightblue', label='Buffered Area')

# plt.title('Line with Buffered Area')
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.legend()
# plt.grid()
# plt.axis('equal')
# plt.show()


# def check_trips_equivalence(close_trips_start, close_trips_end):
#     """
#     Check if close_trips_start and close_trips_end are equivalent.
    
#     Args:
#     close_trips_start (list): List of tuples for start trips
#     close_trips_end (list): List of tuples for end trips
    
#     Returns:
#     bool: True if equivalent, False otherwise
#     """
#     if len(close_trips_start) != len(close_trips_end):
#         print("Lists have different lengths.")
#         return False
    
#     differences = []
#     for i, (start, end) in enumerate(zip(close_trips_start, close_trips_end)):
#         if start != end:
#             differences.append((i, start, end))
    
#     if not differences:
#         print("The lists are identical.")
#         return True
#     else:
#         print(f"Found {len(differences)} differences:")
#         for diff in differences[:10]:  # Print first 10 differences
#             print(f"Index {diff[0]}: Start {diff[1]}, End {diff[2]}")
#         if len(differences) > 10:
#             print(f"... and {len(differences) - 10} more differences.")
#         return False

# # Usage
# are_equivalent = check_trips_equivalence(close_trips_start, close_trips_end)
# print(f"Are the trip lists equivalent? {are_equivalent}")